In [ ]:
def parse_line(s):
    sl = s.rstrip().split(';')
    if len(sl) >= 5:
        return [float(sl[1]) / 1000, float(sl[2]), float(sl[3]), float(sl[4])]
    else:
        print(s)
        return None
    
def oversample(l, v):
    if v != 0:
        for i in range(0, len(l)):
            start = max(0, i + 1 - v)
            l[i] = sum(l[start:i + 1]) / (i + 1 - start)

def kalman(l, err_measure = 1.0, err_estimate = 1.0, q = 0.5):
    err_measure = 1.0
    err_estimate = 1.0
    last_estimate = l[0]
    for i in range(0, len(l)):
        v = l[i]
        kalman_gain = err_estimate / (err_estimate + err_measure)
        current_estimate = last_estimate + kalman_gain * (v - last_estimate)
        err_estimate =  (1.0 - kalman_gain) * err_estimate + abs(last_estimate - current_estimate) * q
        last_estimate = current_estimate;
        l[i] = current_estimate

#with open('wr-1-data-20230218-6-2.txt') as f:
#with open('drop-sd-2023.04.09.txt') as f:
#with open('drop-sd-2023.04.16-17.44.04.txt') as f:
with open('DATA-27.TXT') as f:
    #f.readlines(1900)
    #for i in range(510):
    #    s = f.readline()
    m = list(filter(lambda x: x, map(parse_line, f)))
    
    #m = m[:150]
    t = list(map(lambda x: x[0], m))
    h = list(map(lambda x: x[1], m))
    bat = list(map(lambda x: x[2], m))

h1 = list(h)
h2 = list(h)

oversample(h1, 5)
kalman(h2, q = 0.3)
v = [0]
v1 = [0]
v2 = [0]
vt = []
vtx = []
v11 = [0, 0, 0, 0]
for i, e in enumerate(m[:-1]):
    dh = h[i + 1] - h[i] 
    dh1 = h1[i + 1] - h1[i] 
    dh2 = h2[i + 1] - h2[i] 
    dt = m[i + 1][0] - e[0]
    v.append(dh / dt)
    v1.append(dh1 / dt)
    v2.append(dh2 / dt)
    #vtx.append(tx[i + 1] - tx[i])
    if len(vt) == 0:
        vt.append(dt)
    vt.append(dt)
    if i >= 5:
        dh11 = h1[i] - h1[i - 5] 
        dt = e[0] - m[i - 5][0]
        v11.append(dh11) # / dt)
    
#kalman(v1, q=0.1)
kalman(v2, err_measure = 10.0, err_estimate = 10.0, q=0.2)

import plotly.graph_objects as go
from plotly.subplots import make_subplots

#fig = go.Figure()
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=t, y=h, name='Высота (сырые данные)'))
#fig.add_trace(go.Scatter(x=t, y=h1, name='Высота (усреднение)'))
fig.add_trace(go.Scatter(x=t, y=h2, name='Высота (фильтр Калмана)'))
# fig.add_trace(go.Scatter(x=t, y=h3, name='Высота (фильтр Калмана, МК)'))

#fig.add_trace(go.Scatter(x=t, y=vt, name='Шаг времени'))
#fig.add_trace(go.Scatter(x=t, y=vtx, name='Отправлено байт'))

#fig.add_trace(go.Scatter(x=t, y=v, name='Speed, raw'), secondary_y=True,)
#fig.add_trace(go.Scatter(x=t, y=v1, name='Speed, oversample'), secondary_y=True,)
fig.add_trace(go.Scatter(x=t, y=v2, name='Скорость (фильтр Калмана)'), secondary_y=True,)
#fig.add_trace(go.Scatter(x=t, y=v11, name='Скорость (фильтр Калмана x5)'), secondary_y=True,)

#fig.add_trace(go.Scatter(x=t, y=bat, name='Напряжение'), secondary_y=True,)
#fig.add_trace(go.Scatter(x=t, y=st, name='Серво'), secondary_y=True,)

fig.update_xaxes(title_text="Время, <i>сек</i>")
fig.update_yaxes(title_text="Высота, <i>м</i>", secondary_y=False,)
fig.update_yaxes(title_text="Скорость, <i>м/с</i>", secondary_y=True,)

fig.show()